This notebook assumes that the database has been filtered and has access to this filtered dataset. The notebook performs the relevant denoising and preprocessing on the signals in the database.

In [1]:
%run "Database_Filtering.ipynb" #allowing access to the filtered database

100%|████████████████████████████████████████████████████████████████████████████████| 549/549 [00:13<00:00, 40.09it/s]


In [2]:
from scipy.signal import butter, filtfilt #for the butterworth filter

In [3]:
no_patients = allowed_patients.count_patients()


signals = np.zeros(shape=(no_patients, sample_length))
for i in tqdm(range(0, no_patients)):
    signal = allowed_patients.get_patients(i).read_signal(['v1']).reshape(-1)
    signals[i] = signal

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:02<00:00, 90.74it/s]


In [4]:
# Define the cutoff frequency for the Butterworth filter
cutoff_freq = 1  # Adjust this value based on the characteristics of your signal

# Normalize the cutoff frequency
nyquist_freq = 500  # Nyquist frequency is half the sampling frequency
normalized_cutoff = cutoff_freq / nyquist_freq

#order
order = 3




def butterworth_detrending(signal, cut_off_freq, nyquist_freq, order):
    normalised_cut_off = cut_off_freq/nyquist_freq
    
    b, a = butter(order, normalized_cutoff, btype='low')

    baseline = filtfilt(b, a, signal)#thought this would give the detrended signal but gives a very good fit here
    detrended_signal = signal-baseline
    return detrended_signal


butterworth_detrended_signals = np.zeros(shape=(no_patients, sample_length))
trends = np.zeros(shape=(no_patients, sample_length))

for i, signal in enumerate(signals):
    butterworth_detrended_signals[i] = butterworth_detrending(signal, cutoff_freq, nyquist_freq, order)